In [1]:
"""
Initial Evaluation - Prompting Strategy Comparison
Testing naive RAG with different prompting approaches
"""
import sys
sys.path.insert(0, '../src')

from naive_rag import NaiveRAG
from utils import calculate_f1_score, exact_match
import pandas as pd
import json
import time
from tqdm import tqdm

print("Loading RAG system...")
rag = NaiveRAG()
documents, qa_pairs = rag.load_dataset()
rag.create_embeddings(batch_size=32)
vector_db = rag.build_vector_db()
# Free up RAM by deleting large embeddings array
#del rag.embeddings
#import gc
#gc.collect()
rag.load_generator()

print(f"\nSystem ready")
print(f"Total test questions: {len(qa_pairs)}")

2025-10-02 02:12:58,427 - naive_rag - INFO - Initializing RAG with embedding model: all-MiniLM-L6-v2
2025-10-02 02:12:58,441 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: cpu
2025-10-02 02:12:58,441 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Utils module loaded successfully!
Loading RAG system...


2025-10-02 02:13:00,714 - naive_rag - INFO - RAG system initialized successfully
2025-10-02 02:13:00,714 - naive_rag - INFO - Loading RAG Mini Wikipedia dataset...
2025-10-02 02:13:00,714 - naive_rag - INFO - Loading text corpus...
2025-10-02 02:13:04,803 - naive_rag - INFO - Loading Q&A pairs...
2025-10-02 02:13:06,823 - naive_rag - INFO - Loaded 3200 documents and 918 Q&A pairs
2025-10-02 02:13:06,823 - naive_rag - INFO - Creating embeddings for 3200 documents...


Batches:   0%|          | 0/100 [00:00<?, ?it/s]

2025-10-02 02:14:56,992 - naive_rag - INFO - Created embeddings with shape: (3200, 384)
2025-10-02 02:14:56,992 - naive_rag - INFO - Building FAISS vector database...
2025-10-02 02:14:56,999 - naive_rag - INFO - FAISS index built with 3200 vectors
2025-10-02 02:14:56,999 - naive_rag - INFO - Loading generation model: google/flan-t5-base
2025-10-02 02:14:57,835 - naive_rag - INFO - Generation model loaded successfully



System ready
Total test questions: 918


In [3]:
#Converting the qa_pairs to a list and taking the first 100 questions
test_size = 100
test_questions = [qa_pairs[i] for i in range(min(test_size, len(qa_pairs)))]

print(f"Evaluating on {test_size} questions")
print(f"\nSample Questions:")
for i in range(5):
    print(f"{i+1} Q: {test_questions[i]['question']}")
    print(f"{i+1} A: {test_questions[i]['answer']}")

Evaluating on 100 questions

Sample Questions:
1 Q: Was Abraham Lincoln the sixteenth President of the United States?
1 A: yes
2 Q: Did Lincoln sign the National Banking Act of 1863?
2 A: yes
3 Q: Did his mother die of pneumonia?
3 A: no
4 Q: How many long was Lincoln's formal education?
4 A: 18 months
5 Q: When did Lincoln begin his political career?
5 A: 1832


In [5]:
# Test all 4 prompting strategies
strategies = ["basic", "cot", "persona", "instruction"]
results = []

print("Starting evaluation across all strategies...")
print("This tales ~15-20 minutes\n")

for strategy in strategies:
	print(f"\n")
	print(f"Testing strategy: {strategy.upper()}")
	print("\n")
	
	strategy_results = []
	start_time = time.time()
	
	for i, qa in enumerate(tqdm(test_questions, desc=f"{strategy}")):
		question = qa['question']
		expected_answer = qa['answer']
		
		try:
			# Get RAG answer
			result = rag.query(question, top_k=1, prompt_strategy=strategy)
			predicted_answer = result['answer']
			
			# Calculate metrics
			f1 = calculate_f1_score(predicted_answer, expected_answer)
			em = exact_match(predicted_answer, expected_answer)
			
			strategy_results.append({
				'question': question,
				'expected': expected_answer,
				'predicted': predicted_answer,
				'f1_score': f1,
				'exact_match': em,
				'retrieval_score': result['retrieval_score']
			})
			
		except Exception as e:
			print(f"\nError on question {i}: {e}")
			strategy_results.append({
				'question': question,
				'expected': expected_answer,
				'predicted': "ERROR",
				'f1_score': 0.0,
				'exact_match': False,
				'retrieval_score': 0.0
			})
	
	# Calculate aggregate metrics
	avg_f1 = sum(r['f1_score'] for r in strategy_results) / len(strategy_results)
	avg_em = sum(r['exact_match'] for r in strategy_results) / len(strategy_results)
	elapsed = time.time() - start_time
	
	print(f"\n{strategy.upper()} Results:")
	print(f"  Average F1 Score: {avg_f1:.3f}")
	print(f"  Exact Match Rate: {avg_em:.3f} ({int(avg_em*100)}%)")
	print(f"  Time: {elapsed/60:.1f} minutes")
	
	results.append({
		'strategy': strategy,
		'avg_f1': avg_f1,
		'avg_em': avg_em,
		'time_minutes': elapsed/60,
		'detailed_results': strategy_results
	})

print("\n")
print("EVALUATION COMPLETE")
print("\n")

Starting evaluation across all strategies...
This tales ~15-20 minutes



Testing strategy: BASIC




basic:   0%|                                                                                   | 0/100 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

basic:   1%|▊                                                                          | 1/100 [00:01<02:30,  1.52s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

instruction:  99%|███████████████████████████████████████████████████████████████████▎| 99/100 [01:18<00:00,  1.93it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

instruction: 100%|███████████████████████████████████████████████████████████████████| 100/100 [01:18<00:00,  1.27it/s]


INSTRUCTION Results:
  Average F1 Score: 0.267
  Exact Match Rate: 0.200 (20%)
  Time: 1.3 minutes


EVALUATION COMPLETE




In [13]:
# Analyze CoT strategy failures
cot_results = [r for r in results if r['strategy'] == 'cot'][0]['detailed_results']

# Sort by F1 score to find worst performers
worst_cot = sorted(cot_results, key=lambda x: x['f1_score'])[:20]

print("CoT Strategy - Worst 20 Examples:\n")
for i, result in enumerate(worst_cot, 1):
    print(f"{i}. Q: {result['question']}")
    print(f"   Expected: {result['expected']}")
    print(f"   Got: {result['predicted']}")
    print(f"   F1: {result['f1_score']:.3f}")
    print(f"   Retrieval: {result['retrieval_score']:.3f}\n")

CoT Strategy - Worst 20 Examples:

1. Q: Was Abraham Lincoln the sixteenth President of the United States?
   Expected: yes
   Got: Abraham Lincoln was the 16th President of the United States. Abraham Lincoln was born in 1865. The answer: no.
   F1: 0.000
   Retrieval: 0.710

2. Q: Did Lincoln sign the National Banking Act of 1863?
   Expected: yes
   Got: The relevant sentence in the passage is: Also included was the creation of the system of national banks by the National Banking Acts of 1863, 1864, and 1865, which allowed the creation of a strong national financial system. The answer: yes.
   F1: 0.000
   Retrieval: 0.641

3. Q: Did his mother die of pneumonia?
   Expected: no
   Got: The cause of his continual poor health was never precisely determined, though speculation focuses on tuberculosis, stomach cancer, or a combination of the two. Muir, 103. The answer: no.
   F1: 0.000
   Retrieval: 0.483

4. Q: When did Lincoln begin his political career?
   Expected: 1832
   Got: The r

In [7]:
# Create comparison table
comparison = pd.DataFrame([
	{
		'Strategy': r['strategy'],
		'Avg F1 Score': f"{r['avg_f1']:.3f}",
		'Exact Match %': f"{r['avg_em']*100:.1f}%",
		'Time (min)': f"{r['time_minutes']:.1f}"
	}
	for r in results
])

print("\nPROMPTING STRATEGY COMPARISON")
print(comparison.to_string(index=False))

# Identify best strategy
best_f1 = max(results, key=lambda x: x['avg_f1'])
best_em = max(results, key=lambda x: x['avg_em'])

print(f"\nBest F1 Score: {best_f1['strategy']} ({best_f1['avg_f1']:.3f})")
print(f"Best Exact Match: {best_em['strategy']} ({best_em['avg_em']*100:.1f}%)")


PROMPTING STRATEGY COMPARISON
   Strategy Avg F1 Score Exact Match % Time (min)
      basic        0.444         39.0%        1.8
        cot        0.064          0.0%        8.3
    persona        0.403         35.0%        1.5
instruction        0.267         20.0%        1.3

Best F1 Score: basic (0.444)
Best Exact Match: basic (39.0%)


In [9]:
# Save detailed results
import os
os.makedirs('../results', exist_ok=True)

# Save comparison table
comparison.to_csv('../results/step3_prompting_comparison.csv', index=False)

# Save full results
with open('../results/step3_detailed_results.json', 'w') as f:
	json.dump(results, f, indent=2)

print("Results saved to ../results/")
print("  - step3_prompting_comparison.csv")
print("  - step3_detailed_results.json")

Results saved to ../results/
  - step3_prompting_comparison.csv
  - step3_detailed_results.json


In [11]:
# Analyze where the system fails
best_strategy_results = best_f1['detailed_results']

# Find worst performing questions
sorted_by_f1 = sorted(best_strategy_results, key=lambda x: x['f1_score'])

print("Worst performing questions (lowest F1):")
for i, r in enumerate(sorted_by_f1[:10], 1):
	print(f"\n{i}. Q: {r['question']}")
	print(f"   Expected: {r['expected']}")
	print(f"   Got: {r['predicted']}")
	print(f"   F1: {r['f1_score']:.3f}")

Worst performing questions (lowest F1):

1. Q: Was Abraham Lincoln the sixteenth President of the United States?
   Expected: yes
   Got: no
   F1: 0.000

2. Q: Did his mother die of pneumonia?
   Expected: no
   Got: unanswerable
   F1: 0.000

3. Q: Did Lincoln ever represent Alton & Sangamon Railroad?
   Expected: Yes
   Got: no
   F1: 0.000

4. Q: Which county was Lincoln born in?
   Expected: Hardin County
   Got: Illinois
   F1: 0.000

5. Q: When did Lincoln first serve as President?
   Expected: March 4, 1861
   Got: 1865
   F1: 0.000

6. Q: Who assassinated Lincoln?
   Expected: John Wilkes Booth
   Got: charles dickens
   F1: 0.000

7. Q: Was Lincoln chosen as a presidential candidate in 1860?
   Expected: Yes
   Got: Lincoln was eventually chosen
   F1: 0.000

8. Q: How old was Lincoln in 1816?
   Expected: seven
   Got: 18
   F1: 0.000

9. Q: How long was Lincoln's legal Career?
   Expected: 23 years
   Got: 1837
   F1: 0.000

10. Q: What trail did Lincoln use a Farmers' Alma